In [1]:
!pip install tensorflow==1.6

     |████████████████████████████████| 45.9MB 68kB/s 
     |████████████████████████████████| 3.1MB 39.0MB/s 
     |████████████████████████████████| 890kB 46.9MB/s 
  Created wheel for html5lib: filename=html5lib-0.9999999-cp36-none-any.whl size=107220 sha256=10420557e05b5efbe89e96d2d8e58b5c7d090d96fde5dc7976821a8d5cf10f5a
  Stored in directory: /root/.cache/pip/wheels/50/ae/f9/d2b189788efcf61d1ee0e36045476735c838898eef1cad6e29
Successfully built html5lib
  Found existing installation: html5lib 1.0.1
    Uninstalling html5lib-1.0.1:
      Successfully uninstalled html5lib-1.0.1
  Found existing installation: bleach 3.1.4
    Uninstalling bleach-3.1.4:
      Successfully uninstalled bleach-3.1.4
  Found existing installation: tensorboard 2.2.0
    Uninstalling tensorboard-2.2.0:
      Successfully uninstalled tensorboard-2.2.0
  Found existing installation: tensorflow 2.2.0rc2
    Uninstalling tensorflow-2.2.0rc2:
      Successfully uninstalled tensorflow-2.2.0rc2


# **Sentiment Analysis Using RNN**

In [0]:
import sklearn.datasets
import numpy as np
import re
import collections
from sklearn import metrics

def clearstring(string):
    string = re.sub('[^A-Za-z0-9 ]+', '', string)
    string = string.split(' ')
    string = filter(None, string)
    string = [y.strip() for y in string]
    string = ' '.join(string)
    return string.lower()

def separate_dataset(trainset,ratio=0.5):
    datastring = []
    datatarget = []
    for i in range(int(len(trainset.data)*ratio)):
        data_ = trainset.data[i].split('\n')
        data_ = list(filter(None, data_))
        for n in range(len(data_)):
            data_[n] = clearstring(data_[n])
        datastring += data_
        for n in range(len(data_)):
            datatarget.append(trainset.target[i])
    return datastring, datatarget

def build_dataset(words, n_words):
    count = [['GO', 0], ['PAD', 1], ['EOS', 2], ['UNK', 3]]
    count.extend(collections.Counter(words).most_common(n_words - 1))
    dictionary = dict()
    for word, _ in count:
        dictionary[word] = len(dictionary)
    data = list()
    unk_count = 0
    for word in words:
        index = dictionary.get(word, 0)
        if index == 0:
            unk_count += 1
        data.append(index)
    count[0][1] = unk_count
    reversed_dictionary = dict(zip(dictionary.values(), dictionary.keys()))
    return data, count, dictionary, reversed_dictionary

def str_idx(corpus, dic, maxlen, UNK=3):
    X = np.zeros((len(corpus),maxlen))
    for i in range(len(corpus)):
        for no, k in enumerate(corpus[i].split()[:maxlen][::-1]):
            try:
                X[i,-1 - no]=dic[k]
            except Exception as e:
                X[i,-1 - no]=UNK
    return X

In [3]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:520: FutureWarning: Passing (type, 1) or 

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 451133319805618703]

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [5]:
import tensorflow as tf
import tensorflow
import sklearn
print(tensorflow.__version__)

1.6.0


In [0]:
from sklearn.model_selection import train_test_split

In [0]:
import time,os

In [0]:
trainset = sklearn.datasets.load_files(container_path = '/content/drive/My Drive/sentiment_data',encoding = 'UTF-8')
trainset.data, trainset.target = separate_dataset(trainset,1.0)

In [9]:
print(trainset.target_names)
print(len(trainset.data))
print(len(trainset.target))

['negative', 'positive']
10662
10662


In [0]:
ONEHOT = np.zeros((len(trainset.data),len(trainset.target_names)))
ONEHOT[np.arange(len(trainset.data)),trainset.target] = 1.0
train_x, test_x, train_y, test_y, train_onehot,test_onehot = train_test_split(trainset.data, trainset.target,ONEHOT,test_size=0.2)

In [11]:
concat = ' '.join(trainset.data).split()
vocabulary_size = len(list(set(concat)))
data, count, dictionary, rev_dictionary= build_dataset(concat, vocabulary_size)
print('vocab from size: %d'%(vocabulary_size))
print('Most common words', count[4:10])
print('Sample data ',data[:10], [rev_dictionary[i] for i in data[:10]])

vocab from size: 20465
Most common words [('the', 10129), ('a', 7312), ('and', 6199), ('of', 6063), ('to', 4233), ('is', 3378)]
Sample data  [4, 645, 9, 2540, 8, 22, 4, 3242, 10286, 97] ['the', 'rock', 'is', 'destined', 'to', 'be', 'the', '21st', 'centurys', 'new']


In [0]:
GO = dictionary['GO']
PAD = dictionary['PAD']
EOS = dictionary['EOS']
UNK = dictionary['UNK']
size_layer = 128
num_layers = 2
embedded_size = 128
dimension_output = len(trainset.target_names)
learning_rate = 1e-3
maxlen = 50
batch_size = 128

In [0]:
class Model:
    def __init__(self, size_layer, num_layers, embedded_size,
                 dict_size, dimension_output, learning_rate):
        
        def cells(reuse=False):
            return tf.nn.rnn_cell.BasicRNNCell(size_layer,reuse=reuse)
        
        self.X = tf.placeholder(tf.int32, [None, None])
        self.Y = tf.placeholder(tf.float32, [None, dimension_output])
        
        encoder_embeddings = tf.Variable(tf.random_uniform([dict_size, embedded_size], -1, 1))
        encoder_embedded = tf.nn.embedding_lookup(encoder_embeddings, self.X)
        
        rnn_cells = tf.nn.rnn_cell.MultiRNNCell([cells() for _ in range(num_layers)])
        outputs, _ = tf.nn.dynamic_rnn(rnn_cells, encoder_embedded, dtype = tf.float32)
        
        W = tf.get_variable('w',shape=(size_layer, dimension_output),initializer=tf.orthogonal_initializer())
        b = tf.get_variable('b',shape=(dimension_output),initializer=tf.zeros_initializer())
        
        
        self.logits = tf.matmul(outputs[:, -1], W) + b
        self.cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = self.logits, labels = self.Y))
        self.optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate).minimize(self.cost)
        
        correct_pred = tf.equal(tf.argmax(self.logits, 1), tf.argmax(self.Y, 1))
        self.accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [14]:
tf.reset_default_graph()
sess = tf.InteractiveSession()
model = Model(size_layer,num_layers,embedded_size,vocabulary_size+4,dimension_output,learning_rate)
sess.run(tf.global_variables_initializer())

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See tf.nn.softmax_cross_entropy_with_logits_v2.



In [0]:
saver = tf.train.Saver(tf.global_variables(), max_to_keep=2)
checkpoint_dir = os.path.abspath(os.path.join('./', "checkpoints_basic_rnn"))
checkpoint_prefix = os.path.join(checkpoint_dir, "model")

In [16]:
EARLY_STOPPING, CURRENT_CHECKPOINT, CURRENT_ACC, EPOCH = 5, 0, 0, 0
while True:
    lasttime = time.time()
    if CURRENT_CHECKPOINT == EARLY_STOPPING:
        print('break epoch:%d\n'%(EPOCH))
        break
        
    train_acc, train_loss, test_acc, test_loss = 0, 0, 0, 0
    for i in range(0, (len(train_x) // batch_size) * batch_size, batch_size):
        batch_x = str_idx(train_x[i:i+batch_size],dictionary,maxlen)
        acc, loss, _ = sess.run([model.accuracy, model.cost, model.optimizer], 
                           feed_dict = {model.X : batch_x, model.Y : train_onehot[i:i+batch_size]})
        train_loss += loss
        train_acc += acc
    
    for i in range(0, (len(test_x) // batch_size) * batch_size, batch_size):
        batch_x = str_idx(test_x[i:i+batch_size],dictionary,maxlen)
        acc, loss = sess.run([model.accuracy, model.cost], 
                           feed_dict = {model.X : batch_x, model.Y : train_onehot[i:i+batch_size]})
        test_loss += loss
        test_acc += acc
    
    train_loss /= (len(train_x) // batch_size)
    train_acc /= (len(train_x) // batch_size)
    test_loss /= (len(test_x) // batch_size)
    test_acc /= (len(test_x) // batch_size)
    
    if test_acc > CURRENT_ACC:
        print('epoch: %d, pass acc: %f, current acc: %f'%(EPOCH,CURRENT_ACC, test_acc))
        CURRENT_ACC = test_acc
        CURRENT_CHECKPOINT = 0
    else:
        CURRENT_CHECKPOINT += 1
        
    print('time taken:', time.time()-lasttime)
    print('epoch: %d, training loss: %f, training acc: %f, valid loss: %f, valid acc: %f\n'%(EPOCH,train_loss,
                                                                                          train_acc,test_loss,
                                                                                          test_acc))
    path = saver.save(sess, checkpoint_prefix, global_step=EPOCH)
    EPOCH += 1

epoch: 0, pass acc: 0.000000, current acc: 0.501465
time taken: 15.299572467803955
epoch: 0, training loss: 0.706370, training acc: 0.535866, valid loss: 0.739801, valid acc: 0.501465

time taken: 14.949910402297974
epoch: 1, training loss: 0.619355, training acc: 0.661458, valid loss: 0.824721, valid acc: 0.499512

time taken: 14.770671367645264
epoch: 2, training loss: 0.505822, training acc: 0.754261, valid loss: 1.012179, valid acc: 0.492676

time taken: 15.318742036819458
epoch: 3, training loss: 0.361009, training acc: 0.843868, valid loss: 1.264383, valid acc: 0.499023

time taken: 14.937932968139648
epoch: 4, training loss: 0.236770, training acc: 0.905658, valid loss: 1.780690, valid acc: 0.499023

time taken: 14.862792253494263
epoch: 5, training loss: 0.152105, training acc: 0.943774, valid loss: 2.107150, valid acc: 0.489258

break epoch:6



In [17]:
logits = sess.run(model.logits, feed_dict={model.X:str_idx(test_x,dictionary,maxlen)})
print(metrics.classification_report(test_y, np.argmax(logits,1), target_names = trainset.target_names))



#Predict
checkpoint_file = tf.train.latest_checkpoint(os.path.join('./', 'checkpoints_basic_rnn'))
saver = tf.train.import_meta_graph("{}.meta".format(checkpoint_file))
saver.restore(sess, checkpoint_file)


def predict(sentence):
    logits = sess.run(model.logits, feed_dict={model.X:str_idx([sentence],dictionary,maxlen)})
    return trainset.target_names[np.argmax(logits,1)[0]]


predict('i love this book')

              precision    recall  f1-score   support

    negative       0.62      0.73      0.67      1026
    positive       0.70      0.58      0.63      1107

    accuracy                           0.65      2133
   macro avg       0.66      0.65      0.65      2133
weighted avg       0.66      0.65      0.65      2133

INFO:tensorflow:Restoring parameters from /content/checkpoints_basic_rnn/model-5


'negative'

# **RNN and LSTM**

In [0]:
from sklearn.model_selection import train_test_split
import time

In [19]:
trainset = sklearn.datasets.load_files(container_path='/content/drive/My Drive/sentiment_data',encoding='UTF-8')
trainset.data, trainset.target = separate_dataset(trainset,1.0)
print(trainset.target_names)
print(len(trainset.data))
print(len(trainset.target))

['negative', 'positive']
10662
10662


In [20]:
ONEHOT = np.zeros((len(trainset.data),len(trainset.target_names)))
ONEHOT[np.arange(len(trainset.data)),trainset.target] = 1.0
train_X, test_X, train_Y, test_Y , train_onehot, test_onehot = train_test_split(trainset.data,trainset.target, ONEHOT,test_size=0.2)

concat = ' '.join(trainset.data).split()
vocabulary_size = len(list(set(concat)))
data, count, dictionary, rev_dictionary = build_dataset(concat, vocabulary_size)
print('Vocab from size: %d'%(vocabulary_size))
print('Most Common words',count[4:10])
print('Sample data', data[:10],[rev_dictionary[i] for i in data[:10]])

Vocab from size: 20465
Most Common words [('the', 10129), ('a', 7312), ('and', 6199), ('of', 6063), ('to', 4233), ('is', 3378)]
Sample data [4, 645, 9, 2540, 8, 22, 4, 3242, 10286, 97] ['the', 'rock', 'is', 'destined', 'to', 'be', 'the', '21st', 'centurys', 'new']


In [0]:
GO = dictionary['GO']
PAD = dictionary['PAD']
EOS = dictionary['EOS']
UNK = dictionary['UNK']

In [0]:
size_layer = 128
num_layers = 2
embedded_size = 128
dimension_output = len(trainset.target_names)
learning_rate = 1e-3
maxlen = 50
batch_size = 128

# **Define LSTM Model**

In [0]:
class Model:
  def __init__(self,size_layer,num_layers,embedded_size, dict_size, dimension_output, learning_rate):
    def cells(reuse=False):
      return tf.nn.rnn_cell.LSTMCell(size_layer,initializer=tf.orthogonal_initializer(),reuse=reuse)
    self.X = tf.placeholder(tf.int32, [None,None])
    self.Y = tf.placeholder(tf.float32,[None,dimension_output])
    encoder_embeddings = tf.Variable(tf.random_uniform([dict_size, embedded_size],-1,1))
    encoder_embedded = tf.nn.embedding_lookup(encoder_embeddings, self.X)
    rnn_cells = tf.nn.rnn_cell.MultiRNNCell([cells() for _ in range(num_layers)])
    outputs, _ = tf.nn.dynamic_rnn(rnn_cells, encoder_embedded, dtype =tf.float32)
    W =tf.get_variable('w', shape=(size_layer, dimension_output),initializer=tf.orthogonal_initializer())
    b = tf.get_variable('b',shape=(dimension_output),initializer=tf.zeros_initializer())
    self.logits = tf.matmul(outputs[:,-1],W)+ b
    self.cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=self.logits,labels=self.Y))
    self.optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(self.cost)
    correct_pred = tf.equal(tf.argmax(self.logits,1),tf.argmax(self.Y,1))
    self.accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# **Training**

In [0]:
tf.reset_default_graph()
sess = tf.InteractiveSession()
model = Model(size_layer, num_layers, embedded_size, vocabulary_size+4, dimension_output, learning_rate)
sess.run(tf.global_variables_initializer())
saver = tf.train.Saver(tf.global_variables(),max_to_keep=2)
checkpoint_dir = os.path.abspath(os.path.join('./',"checkpoint_rnn_lstms"))
checkpoint_prefix = os.path.join(checkpoint_dir,"model")

In [25]:
EARLY_STOPPING, CURRENT_CHECKPOINT, CURRENT_ACC, EPOCH = 5, 0, 0, 0
while True:
    lasttime = time.time()
    if CURRENT_CHECKPOINT == EARLY_STOPPING:
        print('break epoch:%d\n'%(EPOCH))
        break
        
    train_acc, train_loss, test_acc, test_loss = 0, 0, 0, 0
    for i in range(0, (len(train_X) // batch_size) * batch_size, batch_size):
        batch_x = str_idx(train_X[i:i+batch_size],dictionary,maxlen)
        acc, loss, _ = sess.run([model.accuracy, model.cost, model.optimizer], 
                           feed_dict = {model.X : batch_x, model.Y : train_onehot[i:i+batch_size]})
        train_loss += loss
        train_acc += acc
    
    for i in range(0, (len(test_X) // batch_size) * batch_size, batch_size):
        batch_x = str_idx(test_X[i:i+batch_size],dictionary,maxlen)
        acc, loss = sess.run([model.accuracy, model.cost], 
                           feed_dict = {model.X : batch_x, model.Y : train_onehot[i:i+batch_size]})
        test_loss += loss
        test_acc += acc
    
    train_loss /= (len(train_X) // batch_size)
    train_acc /= (len(train_X) // batch_size)
    test_loss /= (len(test_X) // batch_size)
    test_acc /= (len(test_X) // batch_size)
    
    if test_acc > CURRENT_ACC:
        print('epoch: %d, pass acc: %f, current acc: %f'%(EPOCH,CURRENT_ACC, test_acc))
        CURRENT_ACC = test_acc
        CURRENT_CHECKPOINT = 0
    else:
        CURRENT_CHECKPOINT += 1
        
    print('time taken:', time.time()-lasttime)
    print('epoch: %d, training loss: %f, training acc: %f, valid loss: %f, valid acc: %f\n'%(EPOCH,train_loss,
                                                                                          train_acc,test_loss,
                                                                                          test_acc))
    path = saver.save(sess, checkpoint_prefix, global_step=EPOCH)
    
    EPOCH += 1

epoch: 0, pass acc: 0.000000, current acc: 0.495605
time taken: 35.01903533935547
epoch: 0, training loss: 0.670577, training acc: 0.573864, valid loss: 0.758578, valid acc: 0.495605

epoch: 1, pass acc: 0.495605, current acc: 0.500000
time taken: 34.75499868392944
epoch: 1, training loss: 0.571715, training acc: 0.696733, valid loss: 0.917919, valid acc: 0.500000

epoch: 2, pass acc: 0.500000, current acc: 0.501465
time taken: 34.7116003036499
epoch: 2, training loss: 0.428541, training acc: 0.799361, valid loss: 1.226348, valid acc: 0.501465

time taken: 34.498682498931885
epoch: 3, training loss: 0.283641, training acc: 0.880563, valid loss: 1.530149, valid acc: 0.496582

epoch: 4, pass acc: 0.501465, current acc: 0.503906
time taken: 34.18601417541504
epoch: 4, training loss: 0.183671, training acc: 0.930398, valid loss: 1.928464, valid acc: 0.503906

time taken: 34.94509530067444
epoch: 5, training loss: 0.112126, training acc: 0.959399, valid loss: 2.385956, valid acc: 0.497559



# **Model Evaluation**

In [26]:
logits = sess.run(model.logits, feed_dict = {model.X: str_idx(test_X, dictionary,maxlen)})
print(metrics.classification_report(test_Y, np.argmax(logits,1),target_names=trainset.target_names))


              precision    recall  f1-score   support

    negative       0.70      0.70      0.70      1030
    positive       0.72      0.73      0.72      1103

    accuracy                           0.71      2133
   macro avg       0.71      0.71      0.71      2133
weighted avg       0.71      0.71      0.71      2133



# **Predict**

In [27]:
checkpoint_file = tf.train.latest_checkpoint(os.path.join('./', 'checkpoints_rnn_lstm'))
saver = tf.train.import_meta_graph("{}.meta".format(checkpoint_file))
saver.restore(sess, checkpoint_file)


def predict(sentence):
    logits = sess.run(model.logits, feed_dict={model.X:str_idx([sentence],dictionary,maxlen)})
    return trainset.target_names[np.argmax(logits,1)[0]]


predict('i love this book')

OSError: ignored